In [54]:
import pandas as pd

In [69]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet

--2024-05-26 10:07:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.212, 18.239.238.152, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1362284 (1.3M) [binary/octet-stream]
Saving to: ‘green_tripdata_2024-01.parquet’

green_tripdata_2024 100%[===================>]   1.30M   937KB/s    in 1.4s    

2024-05-26 10:07:44 (937 KB/s) - ‘green_tripdata_2024-01.parquet’ saved [1362284/1362284]



In [70]:
df = pd.read_parquet('green_tripdata_2024-01.parquet')

In [71]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,N,1.0,236,239,1.0,1.98,12.80,1.0,0.5,3.61,0.0,NaN,1.0,21.66,1.0,1.0,2.75
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,N,1.0,65,170,5.0,6.54,30.30,1.0,0.5,7.11,0.0,NaN,1.0,42.66,1.0,1.0,2.75
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,N,1.0,74,262,1.0,3.08,19.80,1.0,0.5,3.00,0.0,NaN,1.0,28.05,1.0,1.0,2.75
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,N,1.0,74,116,1.0,2.40,14.20,1.0,1.5,0.00,0.0,NaN,1.0,16.70,2.0,1.0,0.00
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,N,1.0,74,243,1.0,5.14,22.60,1.0,0.5,6.28,0.0,NaN,1.0,31.38,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56546,2,2024-01-31 20:46:00,2024-01-31 20:55:00,None,NaN,33,25,NaN,0.00,11.58,0.0,0.0,3.14,0.0,NaN,1.0,15.72,NaN,NaN,NaN
56547,2,2024-01-31 21:06:00,2024-01-31 21:11:00,None,NaN,72,72,NaN,0.49,11.58,0.0,0.0,0.00,0.0,NaN,1.0,12.58,NaN,NaN,NaN
56548,2,2024-01-31 21:36:00,2024-01-31 21:40:00,None,NaN,72,72,NaN,0.52,11.58,0.0,0.0,2.52,0.0,NaN,1.0,15.10,NaN,NaN,NaN
56549,2,2024-01-31 22:45:00,2024-01-31 22:51:00,None,NaN,41,42,NaN,1.17,14.22,0.0,0.0,0.00,0.0,NaN,1.0,15.22,NaN,NaN,NaN


In [72]:
df_head = df.head(100)

In [73]:
len(df_head)

100

In [74]:
print(pd.io.sql.get_schema(df_head_one, name='green_tripdata'))

CREATE TABLE "green_tripdata" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" REAL,
  "trip_distance" REAL,
  "RatecodeID" REAL,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL,
  "Airport_fee" REAL
)


In [75]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import types

In [76]:
#!wget https://jdbc.postgresql.org/download/postgresql-42.7.3.jar

In [77]:
conf = SparkConf()
conf.set('spark.jars', 'postgresql-42.7.3.jar')

In [78]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('data_ingestion_to_postgres') \
        .config(conf=conf) \
        .config('spark.executor.memory', '1g') \
        .config('spark.executor.cores', '1') \
        .config('spark.cores.max', '1') \
        .enableHiveSupport() \
        .getOrCreate()


In [79]:
spark

In [80]:
spark_df = spark.read.options(header='true').parquet('green_tripdata_2024-01.parquet')

In [81]:
spark_df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2024-01-01 00:46:55|  2024-01-01 00:58:25|                 N|         1|         236|         239|              1|         1.98|       12.8|  1.0|    0.

In [82]:
spark_df.write.format("jdbc") \
        .options(url="jdbc:postgresql://172.20.0.2:5432/newyorktaxi",
                 dbtable="public.green_tripdata",
                 user="root",
                 password="root",
                 driver="org.postgresql.Driver") \
        .mode('overwrite') \
        .save()